In [1]:
!pip install ultralytics supervision -q
from ultralytics import YOLO
import cv2, pandas as pd, numpy as np
from pathlib import Path
import supervision as sv
import matplotlib.pyplot as plt
%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 15.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/capsule_yolo.zip" -d "/content/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import os

# Define the base directory for the labels
labels_base_dir = '/content/capsule_yolo/labels'

def correct_label_files(directory):
    count_changed_files = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                filepath = os.path.join(root, file)
                with open(filepath, 'r') as f:
                    lines = f.readlines()

                modified_lines = []
                changed = False
                for line in lines:
                    # Assuming YOLO format where class_id is the first number
                    parts = line.strip().split(' ', 1)
                    if len(parts) > 0 and parts[0] == '1':
                        modified_lines.append('0 ' + parts[1] + '\n')
                        changed = True
                    else:
                        modified_lines.append(line)

                if changed:
                    with open(filepath, 'w') as f:
                        f.writelines(modified_lines)
                    count_changed_files += 1
    return count_changed_files

print("Correcting labels in training set...")
train_labels_path = os.path.join(labels_base_dir, 'train')
changed_train_files = correct_label_files(train_labels_path)
print(f"Corrected {changed_train_files} training label files.")

print("Correcting labels in validation set...")
val_labels_path = os.path.join(labels_base_dir, 'val')
changed_val_files = correct_label_files(val_labels_path)
print(f"Corrected {changed_val_files} validation label files.")

print("\nLabel correction complete. Please re-run the YOLO training cell (cell `e1b8872d`) now.")

Correcting labels in training set...
Corrected 0 training label files.
Correcting labels in validation set...
Corrected 109 validation label files.

Label correction complete. Please re-run the YOLO training cell (cell `e1b8872d`) now.


In [9]:
import yaml
import os

# Define the base directory for the unzipped dataset
dataset_base_dir = '/content/capsule_yolo'

# Define the content for data_capsule.yaml with absolute paths
data_yaml_content = {
    'train': os.path.join(dataset_base_dir, 'images', 'train'),
    'val': os.path.join(dataset_base_dir, 'images', 'val'),
    'nc': 1,  # Number of classes
    'names': ['defect'] # Class names
}

# Define the path where the yaml file will be saved (in the current working directory)
yaml_file_path = 'data_capsule.yaml'

# Write the content to the yaml file
with open(yaml_file_path, 'w') as file:
    yaml.dump(data_yaml_content, file, default_flow_style=False)

print(f"Corrected '{yaml_file_path}' has been created successfully.")

# Display the content of the created YAML file
with open(yaml_file_path, 'r') as file:
    print(file.read())




Corrected 'data_capsule.yaml' has been created successfully.
names:
- defect
nc: 1
train: /content/capsule_yolo/images/train
val: /content/capsule_yolo/images/val



In [13]:

!yolo segment train \
    data=data_capsule.yaml \
    model=yolov8m-seg.pt \
    epochs=20 \
    imgsz=640 \
    batch=16 \
    patience=25 \
    project=capsule_training \
    name=best_run \
    exist_ok=True

print("ENTRAÎNEMENT TERMINÉ !")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data_capsule.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=best_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0.0, plots=True, pose=12.0, p

In [ ]:
model = YOLO("capsule_runs/final_train/weights/best.pt")

# Test on one defective capsule
test_img = "capsule_yolo/images/val/crack_000.png"
results = model(test_img, save=True, conf=0.25)[0]

# Show result
plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(cv2.imread(results.path), cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Perfect defect segmentation – your model works!")
plt.show()